In [16]:
import os

import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [17]:
def recoverImage(img, t, A=np.array([1,1,1])):
    t_0 = 0.1
    J = np.zeros(img.shape, dtype='float64')

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            J[i,j] = ((img[i,j] - A) / max(t[i,j], t_0)) + A
            
    return J

def haze2dehazed(haze_paths, dMap_paths, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    
    for img_path, dm_path in tqdm(zip(haze_paths, dMap_paths)):
            
        img_name = os.path.split(img_path)[1]
        
        img = cv2.imread(img_path)/255
        dmap = cv2.imread(dm_path)/255
        dmap = cv2.cvtColor(dmap.astype('float32'), cv2.COLOR_BGR2GRAY)
        
        tmap = np.clip(np.exp(-dmap), a_min=0, a_max=1)
        dehaze_img = recoverImage(img, tmap)

        # write to disk
        dehaze_img_name = os.path.join(output_dir, img_name)
        cv2.imwrite(dehaze_img_name, dehaze_img*255)


In [18]:
dmap_dir = "dhazy/NYU_GT"
hazy_dir = "dhazy/NYU_Hazy"
output_dir = "dhazy/NYU_dehazed"

In [22]:
dmap_paths = []
hazy_paths = []

for f in os.listdir(hazy_dir):
    hazy_path = os.path.join(hazy_dir, f)
    dmap_path = os.path.join(dmap_dir, f.replace("_Hazy", "_Depth_"))
    
    dmap_paths.append(dmap_path)
    hazy_paths.append(hazy_path)  

In [24]:
hazy_paths[:5]

['dhazy/NYU_Hazy/1349_Hazy.bmp',
 'dhazy/NYU_Hazy/16_Hazy.bmp',
 'dhazy/NYU_Hazy/792_Hazy.bmp',
 'dhazy/NYU_Hazy/829_Hazy.bmp',
 'dhazy/NYU_Hazy/1044_Hazy.bmp']

In [25]:
haze2dehazed(hazy_paths, dmap_paths, output_dir)

1356it [48:56,  2.17s/it]


KeyboardInterrupt: 